# Water flux within the root - Numerical solutions in Dumux

by Timo Koch (?), Daniel Leitner, Andrea Schnepf


In the following we will give some hints how to solve Benchmark 1 & 2, using Dumux. The source code is available at the rosi GIT repository (TODO link).


We base the problem on porousmediumflow/1p/model.hh meaning there is only one phase (water). 

## General remarks

Using Dumux we have to do some C++ programming specifying the exact problem. This is done by calculating the permeability from our parameters (in the RootsParams class in rootsparams.hh), and by defining the boundary conditions and the source term (in the problem class, in rootsproblem.hh).

We set the permeability by (TODO) 

```C++
    Scalar permeability(const Element& element, const SubControlVolume& scv, const ElementSolution& elemSol) const 
    {
    	Scalar mu = Water::liquidViscosity(0.,0.); // temperature, pressure [Pa s] = 1.e-3
        return kz_*mu/(radius_*radius_*M_PI); // [m^2]
    }
```

The definition of boundary conditions is performed by first choosing the type which is either Neumann or Dirichlet, and then define the Neumann flux, or Dirichlet value for the boundary. 

```C++
    /*!
	 * \brief Specifies which kind of boundary condition should be used
	 */
	BoundaryTypes boundaryTypesAtPos(const GlobalPosition &pos) const
	{
		BoundaryTypes bcTypes;
		bcTypes.setAllNeumann(); // default
		if (onUpperBoundary_(pos)) { // top bc
			bcTypes.setAllDirichlet();
		} else if(onLowerBoundary_(pos)) {
			if (scenario_==1) {
				bcTypes.setAllDirichlet();
			} else {
				bcTypes.setAllNeumann();
			}
		}
		return bcTypes;
	}
    
    /*!
     * \brief Evaluate the boundary conditions Neumann
     * E.g. for the mass balance that would the mass flux in \f$ [ kg / (m^2 \cdot s)] \f$.
     */
    ResidualVector neumann(const Element& element,
                           const FVElementGeometry& fvGeometry,
                           const ElementVolumeVariables& elemVolVars,
                           const SubControlVolumeFace& scvf) const
    {
    	Scalar rho = 1.e3; // kg / m^3
    	Scalar g = 9.8; // m/ s^2
		Scalar l = element.geometry().volume(); // length of element (m)
    	const RootsParams<TypeTag>& params = this->spatialParams();
		Scalar r = params.radius(SubControlVolume()); // root radius (m)
	 	Scalar kz = params.axialConductivity(); // (m^5 s / kg) == ( m^4 / (s Pa) )

    	ResidualVector values;

    	values[conti0EqIdx] = rho*g*kz; // m^3 / s
    	values[conti0EqIdx] /= (r*r*M_PI);

    	values[conti0EqIdx] /= rho; // ???? should be *

    	return values[conti0EqIdx];
    }
    
    /*!
	 * \brief Evaluate the boundary conditions for Dirichlet
	 */
	PrimaryVariables dirichletAtPos(const GlobalPosition &pos) const
	{
		Scalar p0 = 0.;
		std::cout << "Dirichlet " << p0 << " [Pa] \n";
		if (onUpperBoundary_(pos)) { // top bc
			return p0_;
		} else if(onLowerBoundary_(pos)) {
			return pL_;
		}
		return PrimaryVariables(p0);
	}   
    
```

```C++
/*!
	 * \brief Evaluate the source term for all phases within a given
	 *        sub-control-volume.
	 *
	 * This is the method for the case where the source term is
	 * potentially solution dependent and requires some quantities that
	 * are specific to the fully-implicit method.
	 *
	 * \param element The finite element
	 * \param fvGeometry The finite-volume geometry
	 * \param elemVolVars All volume variables for the element
	 * \param scv The sub control volume
	 *
	 * For this method, the return parameter stores the conserved quantity rate
	 * generated or annihilate per volume unit. Positive values mean
	 * that the conserved quantity is created, negative ones mean that it vanishes.
	 * E.g. for the mass balance that would be a mass rate in \f$ [ kg / (m^3 \cdot s)] \f$.
	 */
	ResidualVector source(const Element &element,
			const FVElementGeometry& fvGeometry,
			const ElementVolumeVariables& elemVolVars,
			const SubControlVolume &scv) const
	{
		ResidualVector values;
		Scalar rho = 1.e3; // kg / m^3
		Scalar l = element.geometry().volume(); // length of element (m)
		const RootsParams<TypeTag>& params = this->spatialParams();
		Scalar r = params.radius(scv); // root radius (m)
		Scalar phx = elemVolVars[0].pressure(); // kg/m/s^2
		Scalar phs = soilP_; // kg/m/s^2

		values[conti0EqIdx] = kr_ * 2*r*M_PI*l* (phs - phx); // m^3/s
		values[conti0EqIdx] /= (M_PI*r*r*l);
    	values[conti0EqIdx] /= rho;  // ???? should be *

		return values;//
	}   
```    

### Benchmark 1

First we create the geometry by writing a DGF (Dune Grid file) using the following function

In [1]:
import numpy as np

def createDGF_1Droots(filename, nodes, seg):        
    file = open(filename,"w") # write file 

    file.write("DGF\n") 
    file.write('Vertex\n')
    # file.write('parameters 2\n'); 
    for i in range(0,len(nodes)):
        file.write('{:g} {:g} {:g} \n'.format(nodes[i,0], nodes[i,1], nodes[i,2]))
             
    file.write('#\n');
    file.write('Simplex\n'); 
    # file.write('parameters 2\n'); 
    for i in range(0,len(seg)):
        file.write('{:g} {:g} \n'.format(seg[i,0], seg[i,1]));
        
    # not used...        
    file.write('#\nBOUNDARYSEGMENTS\n2 0\n')          
    file.write('3 {:g}\n'.format(len(seg))) 
    file.write('#\nBOUNDARYDOMAIN\ndefault 1\n')
    file.write('#\n')
    
    file.close() 

TODO we might add all the parameters we need for Benchmark 2

We call it to create a straight single root 

In [2]:
    nnz = 100 # resolution
    L = 0.5 # length of single straight root (m)
    
    nodes = np.zeros((nnz,3))    # create grid
    seg = np.zeros(((nnz-1),2), dtype=int) 
    for i in range(1, nnz):
        seg[i-1,0] = i-1
        seg[i-1,1] = i
        nodes[i,:] = [0.,0.,-i*L/(nnz-1)]  
 
    createDGF_1Droots("singleroot.dgf", nodes, seg)

The rest of the parameters of Benchmark 1, we move to the Dumux input file. These parameters are parsed by Dumux, and can be easily retrieved within the C++ code. The input file singleroot.input is given by: 

[TimeLoop]
DtInitial = 1 # [s]
TEnd = 1 # [s]

[Grid]
File = singleroot.dgf

[Parameter]
SoilP = -200 # cm pressure head
Kr = 2.e-9  # radial conductivity (m^2 s / kg) 
Kz = 5.e-13 # axial conductivity (m^5 s / kg)
Radius = 2.e-3 # radius (m)  

[Problem]
Name = benchmark1
EnableGravity = 1

The file holds the following information:
* [TimeLoop] is not important, since it is a static proplem 
* [Grid] is the geometry file we just created, and 
* [Parameter] all the parameter we supply for Benchmark 1 
* [Prolbem] the name of our problem, and we want to enable gravitation